<a href="https://colab.research.google.com/github/mariabandeira/AdministracaoPublica/blob/main/CadUnico/KNN/TrainningPB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import make_scorer, accuracy_score
from imblearn.under_sampling import RandomUnderSampler

In [18]:
treated_data_PB = pd.read_csv('/content/treated_cadUnicoPB.csv', index_col=0)

In [19]:
treated_data_PB.dropna(inplace=True)
treated_data_PB.drop_duplicates(inplace=True)

In [20]:
treated_data_PB.info()

<class 'pandas.core.frame.DataFrame'>
Index: 297045 entries, 53308 to 3954431
Data columns (total 21 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   cd_ibge                        297045 non-null  int64  
 1   estrato                        297045 non-null  int64  
 2   classf                         297045 non-null  int64  
 3   id_familia                     297045 non-null  float64
 4   vlr_renda_media_fam            297045 non-null  float64
 5   cod_local_domic_fam            297045 non-null  float64
 6   qtd_comodos_domic_fam          297045 non-null  float64
 7   qtd_comodos_dormitorio_fam     297045 non-null  float64
 8   cod_material_piso_fam          297045 non-null  float64
 9   cod_material_domic_fam         297045 non-null  float64
 10  cod_agua_canalizada_fam        297045 non-null  float64
 11  cod_abaste_agua_domic_fam      297045 non-null  float64
 12  cod_escoa_sanitario_domic_fam 

In [21]:
treated_data_PB.head()

,cd_ibge,estrato,classf,id_familia,vlr_renda_media_fam,cod_local_domic_fam,qtd_comodos_domic_fam,qtd_comodos_dormitorio_fam,cod_material_piso_fam,cod_material_domic_fam,...,cod_abaste_agua_domic_fam,cod_escoa_sanitario_domic_fam,cod_destino_lixo_domic_fam,cod_iluminacao_domic_fam,cod_calcamento_domic_fam,ind_familia_quilombola_fam,ind_parc_mds_fam,qtde_pessoas,peso.fam,marc_pbf
index,,,,,,,,,,,,,,,,,,,,,
53308,2504009,2,2,58309.0,190.0,1.0,4.0,1.0,5.0,1.0,...,1.0,1.0,1.0,1.0,1.0,2.0,0.0,4,5.337579,0
53310,2504009,2,2,58311.0,70.0,1.0,2.0,1.0,2.0,1.0,...,1.0,2.0,1.0,1.0,1.0,2.0,0.0,1,5.337579,1
53311,2507200,1,2,58312.0,3.0,2.0,3.0,1.0,2.0,1.0,...,4.0,6.0,3.0,1.0,3.0,2.0,205.0,2,5.337949,1
53312,2504009,2,2,58313.0,600.0,1.0,5.0,2.0,5.0,1.0,...,1.0,1.0,1.0,1.0,1.0,2.0,0.0,1,5.337579,0
53314,2503209,2,2,58315.0,394.0,1.0,5.0,1.0,2.0,1.0,...,1.0,3.0,1.0,1.0,1.0,2.0,0.0,2,5.337078,0


In [22]:
list(treated_data_PB.columns)

['cd_ibge',
 'estrato',
 'classf',
 'id_familia',
 'vlr_renda_media_fam',
 'cod_local_domic_fam',
 'qtd_comodos_domic_fam',
 'qtd_comodos_dormitorio_fam',
 'cod_material_piso_fam',
 'cod_material_domic_fam',
 'cod_agua_canalizada_fam',
 'cod_abaste_agua_domic_fam',
 'cod_escoa_sanitario_domic_fam',
 'cod_destino_lixo_domic_fam',
 'cod_iluminacao_domic_fam',
 'cod_calcamento_domic_fam',
 'ind_familia_quilombola_fam',
 'ind_parc_mds_fam',
 'qtde_pessoas',
 'peso.fam',
 'marc_pbf']

In [23]:
columns_to_drop = ['cd_ibge', 'id_familia', 'peso.fam', 'estrato']
treated_data_PB = treated_data_PB.drop(columns=columns_to_drop)

In [24]:
def classificar(x):
    if x == 1:
        return 1
    elif x == 2:
        return 2
    elif x == 3:
        return 3
    elif x == 4:
        return 4
    elif x == 5:
        return 5
    else:
        return 6

treated_data_PB['qtde_pessoas'] = treated_data_PB['qtde_pessoas'].apply(classificar)
treated_data_PB['qtde_pessoas'].value_counts()

,count
qtde_pessoas,
2,78676
3,76202
1,59858
4,50095
5,20778
6,11436


In [25]:
treated_data_PB.describe()

,classf,vlr_renda_media_fam,cod_local_domic_fam,qtd_comodos_domic_fam,qtd_comodos_dormitorio_fam,cod_material_piso_fam,cod_material_domic_fam,cod_agua_canalizada_fam,cod_abaste_agua_domic_fam,cod_escoa_sanitario_domic_fam,cod_destino_lixo_domic_fam,cod_iluminacao_domic_fam,cod_calcamento_domic_fam,ind_familia_quilombola_fam,ind_parc_mds_fam,qtde_pessoas,marc_pbf
count,297045.000000,297045.000000,297045.000000,297045.000000,297045.000000,297045.000000,297045.000000,297045.000000,297045.000000,297045.000000,297045.000000,297045.000000,297045.000000,297045.000000,297045.000000,297045.000000,297045.000000
mean,2.392856,190.969834,1.193998,4.705277,1.793802,2.854460,1.122857,1.172933,1.454150,1.947823,1.389874,1.133909,1.881600,1.996916,18.595385,2.756155,0.645835
std,0.754056,290.979500,0.395428,1.268337,0.672966,1.404578,0.600400,0.378190,0.903657,1.047897,0.832064,0.625080,0.975757,0.055446,61.284866,1.344019,0.478260
min,1.000000,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000
25%,2.000000,17.000000,1.000000,4.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,2.000000,0.000000,2.000000,0.000000
50%,3.000000,65.000000,1.000000,5.000000,2.000000,2.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,2.000000,0.000000,3.000000,1.000000
75%,3.000000,234.000000,1.000000,5.000000,2.000000,5.000000,1.000000,1.000000,1.000000,3.000000,1.000000,1.000000,3.000000,2.000000,0.000000,4.000000,1.000000
max,3.000000,2862.000000,2.000000,18.000000,20.000000,7.000000,8.000000,2.000000,4.000000,6.000000,6.000000,6.000000,3.000000,2.000000,306.000000,6.000000,1.000000


In [26]:
treated_data_PB.info()

<class 'pandas.core.frame.DataFrame'>
Index: 297045 entries, 53308 to 3954431
Data columns (total 17 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   classf                         297045 non-null  int64  
 1   vlr_renda_media_fam            297045 non-null  float64
 2   cod_local_domic_fam            297045 non-null  float64
 3   qtd_comodos_domic_fam          297045 non-null  float64
 4   qtd_comodos_dormitorio_fam     297045 non-null  float64
 5   cod_material_piso_fam          297045 non-null  float64
 6   cod_material_domic_fam         297045 non-null  float64
 7   cod_agua_canalizada_fam        297045 non-null  float64
 8   cod_abaste_agua_domic_fam      297045 non-null  float64
 9   cod_escoa_sanitario_domic_fam  297045 non-null  float64
 10  cod_destino_lixo_domic_fam     297045 non-null  float64
 11  cod_iluminacao_domic_fam       297045 non-null  float64
 12  cod_calcamento_domic_fam      

### Treinando o modelo KNN

In [27]:
# analisando a distribuição da variável alvo (marc_pbf)
target_distribution = treated_data_PB['marc_pbf'].value_counts(normalize=True) * 100

print(round(target_distribution, 2))

marc_pbf
1    64.58
0    35.42
Name: proportion, dtype: float64


In [28]:
# separar X (features) e y (target)
X = treated_data_PB.drop('marc_pbf', axis=1)
y = treated_data_PB['marc_pbf']

# treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# criando uma instância do RandomUnderSampling
rus = RandomUnderSampler(random_state=42, sampling_strategy = 'majority')

# balanceando os dados
X_resampled, y_resampled = rus.fit_resample(X_train, y_train)

print(y_resampled.value_counts())

marc_pbf
0    73668
1    73668
Name: count, dtype: int64


In [29]:
# aplicar padronização nas features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_resampled)
X_test_scaled = scaler.transform(X_test)

print("Primeiros dados escalados (padronizados):")
print(X_train_scaled[:5])

Primeiros dados escalados (padronizados):
[[ 0.82916683  0.86519823 -0.47555747  0.97840329  1.75644702  1.45712256
  -0.19528418 -0.43965204 -0.48510741 -0.86916817 -0.45368299 -0.20480803
  -0.87153126  0.05269602 -0.29181643  0.25709514]
 [ 0.82916683  2.10519503  2.10279527  0.97840329 -1.19226575 -0.64144158
  -0.19528418 -0.43965204  0.64990157  0.09462198 -0.45368299 -0.20480803
  -0.87153126  0.05269602 -0.29181643 -1.24014272]
 [ 0.82916683  0.66569379 -0.47555747  0.97840329  0.28209064 -0.64144158
  -0.19528418 -0.43965204  1.78491056  1.05841214 -0.45368299 -0.20480803
  -0.87153126  0.05269602  3.16393683 -0.49152379]
 [-1.82748477 -0.09549236 -0.47555747  0.19354987  0.28209064 -0.64144158
  -0.19528418 -0.43965204 -0.48510741  1.05841214 -0.45368299 -0.20480803
   1.18886361  0.05269602 -0.29181643  1.00571406]
 [ 0.82916683  1.93024498  2.10279527  0.97840329  1.75644702 -0.64144158
  -0.19528418 -0.43965204  0.64990157  0.09462198  1.99903185 -0.20480803
   1.18886361 

In [30]:
k_values = list(range(1, 31)) # testando valores de 1 a 30

param_grid = {'n_neighbors': k_values}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train_scaled, y_resampled)

best_k = grid_search.best_params_['n_neighbors']
print("Melhor valor de k encontrado:", best_k)

Melhor valor de k encontrado: 28


In [31]:
# Treine o modelo final com o melhor valor de k
final_model = KNeighborsClassifier(n_neighbors=best_k)
final_model.fit(X_train_scaled, y_resampled)

KNeighborsClassifier(n_neighbors=28)

In [32]:
# Faça previsões nos dados de teste
y_pred = final_model.predict(X_test_scaled)

# Imprima o relatório de classificação
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.87      0.74      0.80     31535
           1       0.87      0.94      0.90     57579

    accuracy                           0.87     89114
   macro avg       0.87      0.84      0.85     89114
weighted avg       0.87      0.87      0.87     89114



### Selecionando features com SelectKBests

In [33]:
selector = SelectKBest(score_func=f_classif, k=5)
X_train_selected = selector.fit_transform(X_resampled, y_resampled)
X_test_selected = selector.transform(X_test)

selected_columns = X_train.columns[selector.get_support()]
print("Características selecionadas:", selected_columns)

Características selecionadas: Index(['vlr_renda_media_fam', 'cod_material_piso_fam',
       'cod_escoa_sanitario_domic_fam', 'cod_calcamento_domic_fam',
       'qtde_pessoas'],
      dtype='object')


In [34]:
k_values = list(range(1, 31)) # testando valores de 1 a 30

param_grid = {'n_neighbors': k_values}

knn = KNeighborsClassifier()

grid_search = GridSearchCV(knn, param_grid, cv=5, scoring='accuracy')

grid_search.fit(X_train_selected, y_resampled)

best_k = grid_search.best_params_['n_neighbors']
print("Melhor valor de k encontrado:", best_k)

Melhor valor de k encontrado: 29


In [35]:
# Treine o modelo final com o melhor valor de k
final_model = KNeighborsClassifier(n_neighbors=best_k)
final_model.fit(X_train_selected, y_resampled)

KNeighborsClassifier(n_neighbors=29)

In [36]:
# Faça previsões nos dados de teste
y_pred_selected = final_model.predict(X_test_selected)

# Imprima o relatório de classificação
print(classification_report(y_test, y_pred_selected))

              precision    recall  f1-score   support

           0       0.54      0.90      0.68     31535
           1       0.92      0.58      0.71     57579

    accuracy                           0.70     89114
   macro avg       0.73      0.74      0.69     89114
weighted avg       0.78      0.70      0.70     89114

